In [3]:
using Rx

A __Subject__ is a sort of bridge or proxy that is available in some implementations of reactive frameworks that acts both as an observer and as an Observable. Because it is an observer, it can subscribe to one or more Observables, and because it is an Observable, it can pass through the items it observes by reemitting them, and it can also emit new items.

Because a Subject subscribes to an Observable, it will trigger that Observable to begin emitting items (if that Observable is “cold” — that is, if it waits for a subscription before it begins to emit items). This can have the effect of making the resulting Subject a “hot” Observable variant of the original “cold” Observable.

See also: 
- [ReactiveX: Subject](http://reactivex.io/documentation/subject.html)
- [Introduction to Rx: Subject](http://introtorx.com/Content/v1.0.10621.0/02_KeyTypes.html#Subject)
- [To Use Subject or Not To Use Subject](https://www.davesexton.com/blog/post/To-Use-Subject-Or-Not-To-Use-Subject.aspx)

### Subject execution modes

Rx.jl supports two both synchronous and asynchronous execution modes for any Subject-like object.

In [12]:
sync_subject = make_subject(Int, mode = SYNCHRONOUS_SUBJECT_MODE)

subscription = subscribe!(sync_subject, logger("sync_subject logger"))

next!(sync_subject, 1)

unsubscribe!(subscription)

next!(sync_subject, 2)

[sync_subject logger] Data: 1


Default mode is _ASYNCHRONOUS_SUBJECT_MODE_.

In [14]:
async_subject = make_subject(Int, mode = ASYNCHRONOUS_SUBJECT_MODE); # or simple make_subject(Int)

### Subject as an actor

It is possible to use some Subject as an Actor. This is the only way to share (multicast) a single observable execution between multiple listeners.

In [15]:
source = from(1:5)

subject = make_subject(Int)

subscription1 = subscribe!(subject, logger("1"))
subscription2 = subscribe!(subject, logger("2"))

subscribe!(source, subject)

unsubscribe!(subscription1)
unsubscribe!(subscription2)

[1] Data: 1
[2] Data: 1
[1] Data: 2
[2] Data: 2
[1] Data: 3
[2] Data: 3
[1] Data: 4
[2] Data: 4
[1] Data: 5
[2] Data: 5
[1] Completed
[2] Completed


### Varieties of Subject

There are few varieties of Subject that are designed for particular use cases. Not all of these are available in all implementations, and some implementations use other naming conventions:

#### BehaviorSubject

When an observer subscribes to a BehaviorSubject, it begins by emitting the item most recently emitted by the source Observable (or a seed/default value if none has yet been emitted) and then continues to emit any other items emitted later by the source Observable(s).

In [16]:
bsubject = make_behavior_subject(Int, 1)

subscription1 = subscribe!(bsubject, logger("1"))

next!(bsubject, 2)

subscription2 = subscribe!(bsubject, logger("2"))

next!(bsubject, 3)

unsubscribe!(subscription1)
unsubscribe!(subscription2)

[1] Data: 1
[1] Data: 2
[2] Data: 2
[1] Data: 3
[2] Data: 3


#### ReplaySubject

ReplaySubject emits to any observer all of the items that were emitted by the source Observable(s), regardless of when the observer subscribes.

In [17]:
rsubject = make_replay_subject(Int, 2)

next!(rsubject, 0);

subscription1 = subscribe!(rsubject, logger("1"))

next!(rsubject, 2)

subscription2 = subscribe!(rsubject, logger("2"))

next!(rsubject, 3)

unsubscribe!(subscription1)
unsubscribe!(subscription2)

[1] Data: 0
[1] Data: 2
[2] Data: 0
[2] Data: 2
[1] Data: 3
[2] Data: 3
